In [1]:
import os

In [3]:
os.chdir('../')

In [85]:
import pandas as pd
from tqdm import tqdm
from collections import Counter

from extract import get_tables_from_raw_html, get_float_values_ratio, calc_type_distribution

In [86]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
df = pd.read_csv('test.csv')
df

,id,url,content,update_date
0,1,https://rosstat.gov.ru/folder/10705,"<!DOCTYPE html><html class=""no-js"" lang=""ru""><...",2024-06-08 18:44:41.728379+00:00
1,5,https://rosstat.gov.ru/statistics/accounts,"<!DOCTYPE html><html class=""no-js"" lang=""ru""><...",2024-06-08 18:54:11.685952+00:00
2,8,https://e-disclosure.ru/portal/company.aspx?id...,"<!DOCTYPE html><html browser=""yandexbrowser"" b...",2024-06-08 19:09:05.133379+00:00
3,9,https://cbr.ru/banking_sector/credit/coinfo/f1...,<!DOCTYPE html><html><head><title>DDoS-Guard</...,2024-06-08 20:31:06.579612+00:00
4,11,https://cbr.ru/about/,"<!DOCTYPE html><html style=""--scrollbar-width:...",2024-06-08 20:31:33.659777+00:00
...,...,...,...,...
3831,4815,https://cbr.ru/Content/Document/File/130701/in...,<!DOCTYPE html><html><head><title>DDoS-Guard</...,2024-06-09 05:08:38.432376+00:00
3832,4816,https://cbr.ru/Content/Document/File/132963/in...,<!DOCTYPE html><html><head><title>DDoS-Guard</...,2024-06-09 05:08:47.816091+00:00
3833,4817,https://cbr.ru/Content/Document/File/120920/in...,<!DOCTYPE html><html><head><title>DDoS-Guard</...,2024-06-09 05:08:51.725626+00:00
3834,4818,https://cbr.ru/Content/Document/File/119988/in...,<!DOCTYPE html><html><head><title>DDoS-Guard</...,2024-06-09 05:08:55.901240+00:00


In [104]:
def get_entities_from_html_df(df):
    res = []
    for i, row in tqdm(df.iterrows()):
        # print(row)
        dataframes = get_tables_from_raw_html(row.content, only_df=False)
        for frame in dataframes:
            res.append({'url': row.url, 'frame': frame['data'], 'meta': frame['meta']})
    return res

In [105]:
def deduplicate_entities(tables):
    deduplicated = []
    for i in range(len(tables)):
        is_duplicate = False
        for j in range(len(deduplicated)):
            if tables[i]['frame'].equals(deduplicated[j]['frame']):
                is_duplicate = True
                break
        if not is_duplicate:
            deduplicated.append(tables[i])
    return deduplicated

In [106]:
def calculate_ratios(tables):
    for row in tables:
        type_distr = calc_type_distribution(row['frame'])
        row.update(type_distr)
    return tables

In [107]:
def find_header(df):
    # проходимся по каждой колонке
    # определяем ее тип (float, datetime, str)
    # для флота и дейттайма пытаемся жадно подобрать header_size
    # делаем шаг и проверяем, стало ли больше похоже на правду
    # дальше проводим majority vote по колонкам и ставим хединг
    header_sizes = []
    for col in test_df.columns:
        col_value = test_df[col]
        distr_raw = calc_type_distribution(col_value)
        if distr_raw['str_per_row'] >= 0.5:
            col_dtype = 'str'
            continue
        elif distr_raw['float_val_count'] >= distr_raw['datetime_val_count']:
            col_dtype = 'float'
            col_type = 'float_per_row'
        elif distr_raw['datetime_val_count'] > distr_raw['float_val_count']:
            col_dtype = 'datetime'
            col_type = 'datetime_per_row'
        else:
            continue

        if distr_raw[col_type] == 1:
            header_sizes.append(0)
            continue

        val_per_row = distr_raw[col_type]    
        prev_val_per_row = val_per_row
        for i in range(1, 4):
            distr_i_row_header = calc_type_distribution(col_value.iloc[i:])
            if distr_i_row_header[col_type] <= prev_val_per_row:
                header_sizes.append(i - 1)
                break
            else:
                prev_val_per_row = distr_i_row_header[col_type]
    
    header_size = Counter(header_sizes).most_common(1) # [0][0]
    if header_size:
        header_size = header_size[0][0]
    else:
        header_size = 0
    
    if header_size > 0:
        df.columns = [df.iloc[i, :] for i in range(header_size)]
        df = df.iloc[header_size:]
    return df

In [110]:
entities = get_entities_from_html_df(df)
deduplicated_entities = deduplicate_entities(entities)
entities = calculate_ratios(deduplicated_entities)

3836it [00:39, 98.16it/s] 


In [134]:
entities[0]

{'url': 'https://e-disclosure.ru/portal/company.aspx?id=1210',
 'frame':                                               0  \
 0                  Полное наименование компании   
 1             Сокращенное наименование компании   
 2                              Место нахождения   
 3   Адрес Субъекта раскрытия, указанный в ЕГРЮЛ   
 4                        Адрес (почтовый адрес)   
 5              Дата государственной регистрации   
 6      Номер Государственной регистрации (ОГРН)   
 7                                           ИНН   
 8                      Зарегистрировавший орган   
 9                              ФИО руководителя   
 10                         Телефон руководителя   
 11               Адрес страницы в сети Интернет   
 
                                                     1  
 0           Банк ВТБ (публичное акционерное общество)  
 1                                      Банк ВТБ (ПАО)  
 2   Российская Федерация, 191144, г. Санкт-Петербу...  
 3   191144, г. Санкт-

In [111]:
test_df = entities[15]['frame']

In [142]:
usefull_entities = list(filter(lambda x: x['float_per_row'] > 0.5 and x['datetime_per_row'] > 0.5, entities))

In [157]:
len(usefull_entities)

141

In [158]:
test_df = usefull_entities[8]['frame']
test_df

,0,1,2,3,4,5,6,7
0,Дата установления,"Cтавка, % годовых","Cтавка, % годовых","Cтавка, % годовых","Cтавка, % годовых","Cтавка, % годовых",None,None
1,Дата установления,на 1 день,на 7 дней,на 1 месяц,на 3 месяца,на 6 месяцев,на 12 месяцев,None
2,Дата установления,18.12.2023,"16,00","16,00","16,10",—,—,"16,25"
3,30.10.2023,"15,00","15,00","15,10",—,—,"15,25",None
4,18.09.2023,"13,00","13,00","13,10",—,—,"13,25",None
...,...,...,...,...,...,...,...,...
81,10.06.2008,"6,75","7,25",—,—,—,—,None
82,29.04.2008,"6,50","7,00",—,—,—,—,None
83,04.02.2008,"6,25","6,75",—,"9,75",—,—,None
84,20.05.2003,"6,00","6,50",—,"9,00",—,—,None


In [164]:
pd.read_json(usefull_entities[0]['frame'].to_json())

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
1,1,,2896,1021500000147,ПАО АКБ «1Банк»,ПАО,10.06.1994,ОТЗ,"362040, Республика Северная Осетия-Алания, г.В..."
2,2,,2306,1027700024560,АКБ «Абсолют Банк» (ПАО),ПАО,22.04.1993,,"127051, г. Москва, Цветной бульвар, д. 18"
3,3,,2879,1027700367507,ПАО АКБ «АВАНГАРД»,ПАО,09.06.1994,,"119180, г. Москва, ул. Большая Якиманка, д.1"
4,4,,415,1231600067654,АО Банк «Аверс»,НПАО,25.09.1990,,"420111, г.Казань, ул.Мусы Джалиля, д.3"
...,...,...,...,...,...,...,...,...,...
671,671,,3467,1067711004437,АО КБ «ЮНИСТРИМ»,НПАО,31.05.2006,,"127083, г. Москва, ул. Верхняя Масловка, д. 20..."
672,672,Расчетная НКО,3549-К,1247700338049,РНКО «Юнона Финанс» (ООО),ООО (Паевое),24.04.2024,,"127015, Российская Федерация, город Москва, ул..."
673,673,Расчетная НКО,3541-К,1217700542344,ООО РНКО «ЮСиЭс»,ООО (Паевое),12.11.2021,АНН,"117449, г. Москва, ул. Новочерёмушкинская, д.10"
674,674,,3027,1077711000091,АО «Яндекс Банк»,НПАО,04.08.1994,,"115035, г. Москва, ул. Садовническая, д. 82 ст..."


In [165]:
def jsonify_entity(entity):
    entity['frame'] = entity['frame'].to_json()
    return entity

In [166]:
jsonified_entities = list(map(jsonify_entity, usefull_entities))

In [179]:
import json

with open('../documents.json', 'w') as f:
    
    f.write(json.dumps(jsonified_entities))

In [169]:
with open('../documents.json') as f:
    loaded_json = json.loads(f.read())

In [173]:
pd.read_json(loaded_json[0]['frame'])

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
1,1,,2896,1021500000147,ПАО АКБ «1Банк»,ПАО,10.06.1994,ОТЗ,"362040, Республика Северная Осетия-Алания, г.В..."
2,2,,2306,1027700024560,АКБ «Абсолют Банк» (ПАО),ПАО,22.04.1993,,"127051, г. Москва, Цветной бульвар, д. 18"
3,3,,2879,1027700367507,ПАО АКБ «АВАНГАРД»,ПАО,09.06.1994,,"119180, г. Москва, ул. Большая Якиманка, д.1"
4,4,,415,1231600067654,АО Банк «Аверс»,НПАО,25.09.1990,,"420111, г.Казань, ул.Мусы Джалиля, д.3"
...,...,...,...,...,...,...,...,...,...
671,671,,3467,1067711004437,АО КБ «ЮНИСТРИМ»,НПАО,31.05.2006,,"127083, г. Москва, ул. Верхняя Масловка, д. 20..."
672,672,Расчетная НКО,3549-К,1247700338049,РНКО «Юнона Финанс» (ООО),ООО (Паевое),24.04.2024,,"127015, Российская Федерация, город Москва, ул..."
673,673,Расчетная НКО,3541-К,1217700542344,ООО РНКО «ЮСиЭс»,ООО (Паевое),12.11.2021,АНН,"117449, г. Москва, ул. Новочерёмушкинская, д.10"
674,674,,3027,1077711000091,АО «Яндекс Банк»,НПАО,04.08.1994,,"115035, г. Москва, ул. Садовническая, д. 82 ст..."


In [175]:
def read_entity_from_json(raw_entity):
    raw_entity['frame'] = pd.read_json(raw_entity['frame'])
    return raw_entity

def read_entities_json(path):
    with open(path) as f:
        entities = json.loads(f.read())
    for entity in entities:
        entity = read_entity_from_json(entity)
    return entities

In [178]:
loaded_entities = read_entities_json('../documents.json')

In [152]:
find_header(test_df)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,ISIN,Номер гос. регистрации,Эмитент,Дата погашения,Рыночная цена на ФБ ММВБ*,Срок операций РЕПО,Срок операций РЕПО,Срок операций РЕПО,Срок операций РЕПО,Срок операций РЕПО,...,Срок операций РЕПО,Срок операций РЕПО,Срок операций РЕПО,Механизм **,None,None,None,None,None,None
1,ISIN,до 1 месяца,до 1 месяца,до 1 месяца,Дата погашения,от 1 месяца до 180 дней,от 1 месяца до 180 дней,от 1 месяца до 180 дней,от 181 дня до 1 года,от 181 дня до 1 года,...,None,None,None,None,None,None,None,None,None,None
2,ISIN,Вид дисконта,Вид дисконта,Вид дисконта,Вид дисконта,Вид дисконта,Вид дисконта,Вид дисконта,Вид дисконта,Вид дисконта,...,None,None,None,None,None,None,None,None,None,None
3,ISIN,нач.,Номер гос. регистрации,мин.,Эмитент,макс.,Дата погашения,нач.,Рыночная цена на ФБ ММВБ*,мин.,...,нач.,мин.,макс.,None,None,None,None,None,None,None
4,ISIN,"Дисконт, %","Дисконт, %","Дисконт, %","Дисконт, %","Дисконт, %","Дисконт, %","Дисконт, %","Дисконт, %","Дисконт, %",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,RU000A0JRTL6,46023RMFS,Минфин России,23.07.2026,"92,443",0,"-0,5","0,5",0,"-0,5",...,0,"-0,5","0,5",ОМ,None,None,None,None,None,None
720,RU000A0ZYZ26,52002RMFS,Минфин России,02.02.2028,"85,366",0,"-0,5","0,5",0,"-0,5",...,0,"-0,5","0,5",ОМ,None,None,None,None,None,None
721,RU000A102069,52003RMFS,Минфин России,17.07.2030,"78,645",0,"-0,5","0,5",0,"-0,5",...,0,"-0,5","0,5",ОМ,None,None,None,None,None,None
722,RU000A103MX5,52004RMFS,Минфин России,17.03.2032,"75,8",0,"-0,5","0,5",0,"-0,5",...,0,"-0,5","0,5",ОМ,None,None,None,None,None,None


In [81]:
header_size

1

In [158]:
for entity in entities:
    entity['frame'] = find_header(entity['frame'])

In [159]:
entities

[{'url': 'https://cbr.ru/banking_sector/registers/',
  'frame': Empty DataFrame
  Columns: [, 2024 год, ]
  Index: [],
  'meta': 'Реестры и перечни кредитных организаций | Банк России',
  'float_val_count': 0,
  'datetime_val_count': 0,
  'na_val_count': 2,
  'total_count': 3,
  'rows_total': 1,
  'cols_total': 3,
  'na_ratio': 0.6666666666666666,
  'datetime_per_row': 0.0,
  'float_per_row': 0.0},
 {'url': 'https://cbr.ru/banking_sector/registers/',
  'frame': 0   Январь Февраль      Март
  1   Апрель     Май      Июнь
  2     Июль  Август  Сентябрь
  3  Октябрь  Ноябрь   Декабрь,
  'meta': 'Реестры и перечни кредитных организаций | Банк России',
  'float_val_count': 0,
  'datetime_val_count': 0,
  'na_val_count': 0,
  'total_count': 12,
  'rows_total': 4,
  'cols_total': 3,
  'na_ratio': 0.0,
  'datetime_per_row': 0.0,
  'float_per_row': 0.0},
 {'url': 'https://cbr.ru/banking_sector/registers/',
  'frame': Empty DataFrame
  Columns: [, 2024 год, ]
  Index: [],
  'meta': 'Реестры и пе

In [137]:
len(deduplicated)

240

In [139]:
len(final_df)

3

In [141]:
final_df[1]['frame'], final_df[1]['url'], final_df[1]['meta']

(       0              1       2              3                          4  \
 0                                                                           
 1      1                   2896  1021500000147            ПАО АКБ «1Банк»   
 2      2                   2306  1027700024560   АКБ «Абсолют Банк» (ПАО)   
 3      3                   2879  1027700367507         ПАО АКБ «АВАНГАРД»   
 4      4                    415  1231600067654            АО Банк «Аверс»   
 ..   ...            ...     ...            ...                        ...   
 671  671                   3467  1067711004437           АО КБ «ЮНИСТРИМ»   
 672  672  Расчетная НКО  3549-К  1247700338049  РНКО «Юнона Финанс» (ООО)   
 673  673  Расчетная НКО  3541-К  1217700542344           ООО РНКО «ЮСиЭс»   
 674  674                   3027  1077711000091           АО «Яндекс Банк»   
 675  675                   2564  1027600000075   ИКБР «ЯРИНТЕРБАНК» (ООО)   
 
                 5           6    7  \
 0                     

In [68]:
from extract import calc_type_distribution

In [81]:
calc_type_distribution(final_df[3]['frame'])

(10, 0, 20)

In [62]:
get_float_values_ratio(final_df[3]['frame'])

0.25

In [48]:
deduplicated[10]['frame']

,0,1,2
0,Дата публикации,Время публикации,Информация - показатель / период наблюдения
1,10.06.2024,—,Средневзвешенные процентные ставки по кредитам...
2,10.06.2024,—,Сведения о средних арифметических процентных с...
3,11.06.2024,—,Средневзвешенные процентные ставки по кредитам...
4,11.06.2024,—,Сведения о размещенных средствах (информация о...
...,...,...,...
201,06.09.2024,—,Краткий комментарий «Процентные ставки по кред...
202,06.09.2024,—,Краткий комментарий «Кредитование юридических ...
203,06.09.2024,—,Информационный бюллетень «Сведения о рынке ипо...
204,06.09.2024,11:00,Денежная база в узком определении (Еженедельны...


In [27]:
res[0]['dataframes'][0].equals(res[0]['dataframes'][2])

True

In [22]:
for url_row in res:
    for df in url_row['dataframes']:
        print(df)
# res[0]['dataframes'][0]

  0         1 2
0    2024 год  
         0        1         2
0   Январь  Февраль      Март
1   Апрель      Май      Июнь
2     Июль   Август  Сентябрь
3  Октябрь   Ноябрь   Декабрь
  0         1 2
0    2024 год  
         0        1         2
0   Январь  Февраль      Март
1   Апрель      Май      Июнь
2     Июль   Август  Сентябрь
3  Октябрь   Ноябрь   Декабрь
  0         1 2
0    2024 год  
         0        1         2
0   Январь  Февраль      Март
1   Апрель      Май      Июнь
2     Июль   Август  Сентябрь
3  Октябрь   Ноябрь   Декабрь
  0         1 2
0    2024 год  
         0        1         2
0   Январь  Февраль      Март
1   Апрель      Май      Июнь
2     Июль   Август  Сентябрь
3  Октябрь   Ноябрь   Декабрь
  0         1 2
0    2024 год  
         0        1         2
0   Январь  Февраль      Март
1   Апрель      Май      Июнь
2     Июль   Август  Сентябрь
3  Октябрь   Ноябрь   Декабрь
  0         1 2
0    2024 год  
         0        1         2
0   Январь  Февраль      Мар

In [16]:
len(get_tables_from_raw_html(df['content'].iloc[0]))

4